# Q1. Rows in LanceDB
- How many rows does the lancedb table "notion_pages__homework" have?

In [1]:
import os
# from google.colab import userdata

# os.environ["SOURCES__REST_API__NOTION__API_KEY"] = userdata.get("SOURCES__REST_API__NOTION__API_KEY")
os.environ["SOURCES__REST_API__NOTION__API_KEY"] = "secret_q4QgENGc9JKXgGContzdV6Z6KMTwzLYXWX9VMFi1dlb"

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

In [2]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

In [14]:
from datetime import datetime, timezone

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "json": {
                        "query": "homework",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config, name="employee_handbook")


def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }


@dlt.resource(
    name="employee_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time": {"dedup_sort": "desc"}}
)
def rest_api_notion_incremental(
    last_edited_time=dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z", primary_key=("block_id"))
):
    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not len(block["content"]):
            continue
        yield block


def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="homework_pipeline",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)


load_notion()


2024-07-09 09:35:51,696|[WARNING]|2628|125474931188352|dlt|client.py|detect_paginator:281|Fallback paginator used: SinglePagePaginator at 721dbfb2cca0. Please provide right paginator manually.
2024-07-09 09:35:52,542|[WARNING]|2628|125474931188352|dlt|client.py|detect_paginator:281|Fallback paginator used: SinglePagePaginator at 721dbfb5d120. Please provide right paginator manually.


int64
version
int64
engine_version
string
pipeline_name
string
state
timestamp[us, tz=UTC]
created_at
string
version_hash
string
_dlt_load_id
string
_dlt_id
_dlt_pipeline_state
[]
int64
version
int64
engine_version
timestamp[us, tz=UTC]
inserted_at
string
schema_name
string
version_hash
string
schema
_dlt_version
[]
homework
[{'name': 'block_id', 'nullable': False, 'primary_key': True, 'data_type': 'text'}, {'name': 'block_type', 'data_type': 'text', 'nullable': True}, {'name': 'content', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'dedup_sort': 'desc', 'name': 'last_edited_time', 'data_type': 'timestamp', 'nullable': True}, {'name': 'inserted_at_time', 'data_type': 'timestamp', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
string
load_id
string
schema_name
int64
status
timestamp[us, tz=UTC]
inserted_at
string
schema_version_hash
_dlt_loads
[]
UPLOAD

In [15]:
import lancedb
db = lancedb.connect("./.lancedb")
print(db.table_names())

['notion_pages____dlt_loads', 'notion_pages____dlt_pipeline_state', 'notion_pages____dlt_version', 'notion_pages___dltSentinelTable', 'notion_pages___employee_handbook', 'notion_pages___homework', 'qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [18]:
db_table = db.open_table("notion_pages___homework")

df = db_table.to_pandas()

In [20]:
df.head()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,c69f1ecf-7b02-5810-8286-3f42659ae9d4,"[-0.024265619, 0.04746082, -0.011796436, 0.063...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-09 09:35:52.543479+00:00,1720517750.9997027,+uCFPqWUXKnKVw
1,f2c18ac0-50f5-5b72-a871-dc5a46780353,"[-0.04966163, 0.10853516, -0.009762589, -0.036...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-09 09:35:52.547472+00:00,1720517750.9997027,T3zgJVQqQOXgFQ
2,4553193e-c655-54df-9a33-cfc570bf34d0,"[-0.06316319, 0.17331506, 0.025351718, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-09 09:35:52.547660+00:00,1720517750.9997027,igUPRmdFIhFyEA
3,791be1a1-6c67-530d-87ab-bd9912500ea5,"[-0.10974315, 0.10586077, 0.0032906013, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-09 09:35:52.547819+00:00,1720517750.9997027,NbGS4H3ot7IzRQ
4,a83497f4-922c-5d62-bab1-53804e93c811,"[0.052423332, -0.06457594, 0.065862976, 0.0145...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-09 09:35:52.548009+00:00,1720517750.9997027,68bMxZKe4MnHOw


# Q2. Running the Pipeline: Last edited time
- In the demo, we created an incremental dlt resource rest_api_notion_incremental that keeps track of last_edited_time. What value does it store after you've run your pipeline once? (Hint: you will be able to get this value by performing some aggregation function on the column last_edited_time of the table)

In [23]:
max(df['last_edited_time'])

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

# Q3. Ask the Assistant
- Find out with the help of the AI assistant: how many PTO days are the employees entitled to in a year?

In [24]:
import ollama

In [25]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [26]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

In [27]:
main()

You:  how many PTO days are the employees entitled to in a year?


Assistant: Employees are entitled to 30 days of paid time off (PTO) per year, as mentioned in the handbook. If an employee wants to take PTO, they must send a request through their HRIS and get approval from their manager or HR. Employees can use unused PTO at any time after their first week with the company but cannot transfer it to the next year. 
If you leave our company, we may compensate accrued PTO with your final paycheck according to local law. When the law doesn’t have provisions, we will compensate accrued leave to employees who were not terminated for cause. If an employee is a non-exempt employee and works on a holiday, they will receive their regular hourly rate with a premium for working on that day.
If you are still unclear about any details or would like more information, please contact the HR department directly to get clarification.


KeyboardInterrupt: Interrupted by user